In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pandas as pd


from google.colab import drive
drive.mount('/drive')

%matplotlib inline

plt.style.use('seaborn-whitegrid')
# plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "17"

Mounted at /drive


<ipython-input-1-5a2870983459>:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
from typing import List

class DateFeatures(TransformerMixin, BaseEstimator):

    """DateFeatures transformer."""
    features = [
        "hour",
        "year",
        "day",
        "weekday",
        "month",
        "quarter",
    ]
    def __init__(self):
        """Nothing much to do."""
        super().__init__()
        self.feature_names: List[str] = []

    def get_feature_names(self):
        """Feature names."""
        return self.feature_names

    def transform(self, df: pd.DataFrame):
        """Annotate date features."""
        Xt = []
        for col in df.columns:
            for feature in self.features:
                date_feature = getattr(
                    getattr(
                        df[col], "dt"
                    ), feature
                )
                date_feature.name = f"{col}_{feature}"
                Xt.append(date_feature)

        df2 = pd.concat(Xt, axis=1)
        self.feature_names = list(df2.columns)
        return df2

    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        """No fitting needed."""
        return self

In [ ]:
import numpy as np
from sklearn.base import TransformerMixin, BaseEstimator
from typing import Dict

class CyclicalFeatures(TransformerMixin, BaseEstimator):
    """CyclicalFeatures transformer."""

    def __init__(self, max_vals: Dict[str, float] = {}):
        """Nothing much to do."""
        super().__init__()
        self.feature_names: List[str] = []
        self.max_vals = max_vals

    def get_feature_names(self):
        """Feature names."""
        return self.feature_names

    def transform(self, df: pd.DataFrame):
        """Annotate date features."""
        Xt = []
        for col in df.columns:
            if col in self.max_vals:
                max_val = self.max_vals[col]
            else:
                max_val = df[col].max()
            for fun_name, fun in [("cos", np.cos), ("sin", np.sin)]:
                date_feature = fun(2 * np.pi * df[col] / max_val)
                date_feature.name = f"{col}_{fun_name}"
                Xt.append(date_feature)

        df2 = pd.concat(Xt, axis=1)
        self.feature_names = list(df2.columns)
        return df2
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        """No fitting needed."""
        return self

In [ ]:

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
#from sklearn import linear_model

preprocessor = ColumnTransformer(
    transformers=[(
        "date",
        make_pipeline(
            DateFeatures(),
            ColumnTransformer(transformers=[
                ("cyclical", CyclicalFeatures(),
                  ["date_day", "date_weekday", "date_month"]
                )
            ], remainder="passthrough")
        ), ["date"],
  ),], remainder="passthrough"
)

preprocess_pipeline = Pipeline(
    [
        ("preprocessing", preprocessor),
        #("clf", linear_model.LinearRegression(),),
    ]
)

In [ ]:
def split_column(dataframe, column_name, value_column):
    global unique_names
    # Extract unique values from the column
    unique_names = dataframe[column_name].unique()

    # Create new columns based on unique values
    for name in unique_names:
        dataframe[name] = dataframe[dataframe[column_name]==name][value_column]
    # Drop the original column
    dataframe = dataframe.drop(columns=[column_name,value_column]).drop_duplicates()

    return dataframe

In [ ]:
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso

model_pipeline = Pipeline(
    [
        ("preprocessing", preprocessor),
        # ("xgb", XGBRegressor(objective="reg:squarederror", n_estimators=1000))
        ("ridge", Ridge(alpha=0.1, max_iter=2000))
        # ("lasso", Lasso(alpha=0.01, max_iter=1500))
    ])

In [ ]:
# Prepare OECD CPI Data
cpi_oecd = pd.read_csv('/drive/My Drive/CPI_OECD.csv')
cpi_oecd = cpi_oecd[cpi_oecd['FREQUENCY'] == 'M']
cpi_oecd = cpi_oecd[(cpi_oecd['Country'] == 'South Africa') & (cpi_oecd['Measure'] == 'National Index')]
cpi_oecd['date'] = pd.to_datetime(cpi_oecd['Time'] +'-01', format='%b-%y-%d')
df = cpi_oecd[['date','Subject','Value']].set_index('date', drop=False)

# Split the 'subject' column into multiple columns and use 'Value' column as corresponding values
# df = split_column(df, 'Subject', 'Value')

# for cols in df.columns:
#   if cols != 'date':
#     df[cols] = ((df[cols] / df[cols].shift(1))-1).dropna()
# df = df.dropna()


# df = df.merge(carSalesFeats,right_index=True,left_index=True,how='outer').dropna()
# df['date'] = df.index

<ipython-input-7-45842d50a89b>:2: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  cpi_oecd = pd.read_csv('/drive/My Drive/CPI_OECD.csv')
<ipython-input-7-45842d50a89b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpi_oecd['date'] = pd.to_datetime(cpi_oecd['Time'] +'-01', format='%b-%y-%d')


In [ ]:
# Prepare CPI data from Zindi
cpi_mar = pd.read_csv('/drive/My Drive/CPI_Historic_Values_Zindi_Mar_23.csv')
cpi_mar['date'] = pd.to_datetime( '01-' + cpi_mar['Month'].str[-7:], format='%d-%m-%Y')
cpi_mar = cpi_mar.rename(columns={'Category':'Subject'})
df = cpi_mar[['date','Subject','Value']].set_index('date',drop=False)

# df = split_column(df,'Subject','Value')

# df = df.merge(carSalesFeats,right_index=True,left_index=True,how='outer').dropna()
# df['date'] = df.index

In [ ]:
# Prepare vehicle data from Zindi
carSales = pd.read_csv('/drive/My Drive/Naamsa_Vehicle_Sales.csv')
carSales['date'] = pd.to_datetime( '01-'+carSales['Month'], format='%d-%y-%b')
df = carSales.set_index('date',drop=False)

excluded_column = 'Month'

# Get column names excluding the specified column
unique_names = [col for col in df.columns if col != excluded_column]
car_feats = [col for col in df.columns if col != excluded_column]

In [ ]:
# Prepare vehicle data from Zindi
petrolPrices = pd.read_excel('/drive/My Drive/pedroldata.xlsx')
petrolPrices['date'] = pd.to_datetime( petrolPrices['date'], format='%y-%b-%d')
df = petrolPrices.set_index('date',drop=False)

excluded_column = 'date'

# Get column names excluding the specified column
unique_names = [col for col in df.columns if col != excluded_column]
car_feats = [col for col in df.columns if col != excluded_column]
col_type = 'feature'


In [ ]:
MAX_HORIZON = 2
X_test_horizon = pd.Series(pd.date_range(
    start=df.index.min(),
    periods=len(df) + MAX_HORIZON,
    freq='MS',
    name="date"
)).reset_index()

In [ ]:
validation_predictions = []
validation_actuals = []

FEATURE_COLS = ['date']
start = '2022-10-01'
split = '2023-04-01'

subjects = ['CPI: 01 - Food and non-Alcoholic beverages']

for sub in df['Subject'].unique():
# for sub in subjects:
    df_temp = df[df['Subject'] == sub]

    MAX_HORIZON = 2
    X_test_horizon = pd.Series(pd.date_range(
        start=pd.to_datetime(start),
        periods=9,
        freq='MS',
        name = 'date'
    )).reset_index()

    TRAIN_SIZE = (pd.to_datetime(split) - df_temp.date.min()).days
    HORIZON = 0
    TARGET_COL = 'Value'

    X_train, X_test = df_temp.iloc[HORIZON:TRAIN_SIZE], df_temp.iloc[TRAIN_SIZE+HORIZON:]
    y_train = df_temp.shift(periods=HORIZON).iloc[HORIZON:TRAIN_SIZE][TARGET_COL]
    y_test = df_temp.shift(periods=HORIZON).iloc[TRAIN_SIZE+HORIZON:][TARGET_COL]

    reg = model_pipeline.fit(X_train[FEATURE_COLS], y_train)

    forecasted = pd.concat([pd.Series(model_pipeline.predict(X_test_horizon[FEATURE_COLS])), pd.Series(X_test_horizon.date)], axis = 1)
    forecasted.columns = [TARGET_COL, 'date']
    forecasted['Subject'] = sub

    actual = pd.concat([pd.Series(df_temp[TARGET_COL]), pd.Series(df_temp.date)], axis = 1)
    actual.columns = [TARGET_COL, 'date']
    actual['Subject'] = sub

    validation_predictions.append(forecasted)
    validation_actuals.append(actual)

validation_predictions = pd.concat(validation_predictions)
validation_actuals = pd.concat(validation_actuals)
validation_actuals = validation_actuals[validation_actuals['date']>= start]



In [ ]:
val_preds = validation_predictions.groupby([validation_predictions['date'].dt.strftime('%y-%m-%d'),'Subject'])['Value'].sum()
val_acts = validation_actuals.groupby([validation_actuals['date'].dt.strftime('%y-%m-%d'),'Subject'])['Value'].sum()

results = pd.concat([val_acts,val_preds], axis = 1)
results.columns = ['act_values', 'pred_values']
results['delta'] = results['pred_values'] - results['act_values']
results['delta_%'] = (results['delta'] / results['act_values'] *100)

print('results as of: ' + str(X_train.date.max()))
results.round(1)

results.to_csv('April_results.csv')
results

results as of: 2023-03-01 00:00:00


act_values  pred_values     delta  \
date     Subject                                                               
22-10-01 Alcoholic beverages and tobacco        106.1   105.588779 -0.511221   
         Clothing and footwear                  102.6   102.466316 -0.133684   
         Communication                           99.8    99.465412 -0.334588   
         Education                              104.4   103.791198 -0.608802   
         Food and non-alcoholic beverages       111.3   111.056482 -0.243518   
...                                               ...          ...       ...   
23-06-01 Housing and utilities                    NaN   105.468848       NaN   
         Miscellaneous goods and services         NaN   108.316758       NaN   
         Recreation and culture                   NaN   104.542184       NaN   
         Restaurants and hotels                   NaN   109.732299       NaN   
         Transport                                NaN   119.592820       NaN   

                                            delta_%  
date     Subject                                     
22-10-01 Alcoholic beverages and tobacco  -0.481830  
         Clothing and footwear            -0.130296  
         Communication                    -0.335258  
         Education                        -0.583144  
         Food and non-alcoholic beverages -0.218795  
...                                             ...  
23-06-01 Housing and utilities                  NaN  
         Miscellaneous goods and services       NaN  
         Recreation and culture                 NaN  
         Restaurants and hotels                 NaN  
         Transport                              NaN  

[117 rows x 4 columns]

In [ ]:
validation_predictions = []
validation_actuals = []

for target in unique_names:
  if col_type =='feature':
    FEATURE_COLS = ['date']
  else:
    FEATURE_COLS = car_feats

  date_features = preprocess_pipeline.fit_transform(df[FEATURE_COLS])
  TRAIN_SIZE = int(len(df) * 1)
  HORIZON = 0
  TARGET_COL = target

  X_train, X_test = df.iloc[HORIZON:TRAIN_SIZE], df.iloc[TRAIN_SIZE+HORIZON:]
  y_train = df.shift(periods=HORIZON).iloc[HORIZON:TRAIN_SIZE][TARGET_COL]
  y_test = df.shift(periods=HORIZON).iloc[TRAIN_SIZE+HORIZON:][TARGET_COL]

  model_pipeline.fit(X_train[FEATURE_COLS], y_train)

  forecasted = pd.concat([pd.Series(model_pipeline.predict(X_test_horizon[FEATURE_COLS])), pd.Series(X_test_horizon.date)],axis=1)
  forecasted.columns = [TARGET_COL, "date"]

  actual = pd.concat([pd.Series(df[TARGET_COL]), pd.Series(df.date)],axis=1)
  actual.columns = [TARGET_COL, "date"]

  validation_predictions.append(forecasted)
  validation_actuals.append(actual)

In [ ]:
def feat_prep(df):
  df = pd.concat(validation_predictions, axis = 1)
  df = df.set_index(df.iloc[:,1])
  df = df.drop(columns='date')
  return df

In [ ]:
petrolPriceFeats = feat_prep(validation_predictions)

In [ ]:
X_test_horizon = X_test_horizon.merge(carSalesFeats, on='date')

In [ ]:
for i, df in enumerate(validation_predictions):
  df['CPI'] = df.columns[0]
  df.rename(columns={df.columns[0]: 'value'}, inplace=True)

validation_predictions = pd.concat(validation_predictions)

In [ ]:
# -------------------------------------
# CREATE AND FORMAT SUBMISSION FILE

cpi_dict = {
    'CPI: 01-12 - All items':'headline CPI',
    'CPI: 01 - Food and non-Alcoholic beverages':'food and non-alcoholic beverages',
    'CPI: 02 - Alcoholic beverages, tobacco and narcotics':'alcoholic beverages and tobacco',
    'CPI: 03 - Clothing and footwear':'clothing and footwear',
    'CPI: 04 - Housing, water, electricity, gas and other fuels':'housing and utilities',
    'CPI: 05 - Furnishings, household equipment and routine household maintenance':'household contents and services',
    'CPI: 06 - Health':'health',
    'CPI: 07 - Transport':'transport',
    'CPI: 08 - Communication':'communication',
    'CPI: 09 - Recreation and culture':'recreation and culture',
    'CPI: 10 - Education':'education',
    'CPI: 11 - Restaurants and hotels':'restaurants and hotels',
    'CPI: 12 - Miscellaneous goods and services':'miscellaneous goods and services'
    }

weights_2023 = {'food and non-alcoholic beverages': 0.1714, 'alcoholic beverages and tobacco': 0.0626, 'clothing and footwear': 0.0365, 'housing and utilities': 0.2449, 'household contents and services': 0.0437, 'health': 0.0144, 'transport': 0.1435, 'communication': 0.0242, 'recreation and culture': 0.052, 'education': 0.0262, 'restaurants and hotels': 0.0325, 'miscellaneous goods and services': 0.1481}

prediction_month = '2023-04-01'
date_obj = datetime.strptime(prediction_month, '%Y-%m-%d')

submission_values = validation_predictions[validation_predictions['date'] == prediction_month]

# Sort by CPI number
submission_values = submission_values[['CPI','value']].sort_values(by=['CPI'],ascending=True)

if submission_values['CPI'].iloc[0] == 'CPI: 01 - Food and non-Alcoholic beverages':
  # After sorting, calculate the headline CPI by taking the sum of products of values and weights for each subject
  not_headline = submission_values[submission_values['CPI'] != 'CPI: 01-12 - All items']
  not_headline['CPI'] = not_headline['CPI'].map(cpi_dict)
  not_headline['weights'] = not_headline['CPI'].map(weights_2023)
  headline = pd.DataFrame(data={'CPI': ['headline CPI'], 'value': [not_headline.apply(lambda row: (row['value']*row['weights']),axis=1).sum()]})
  submission_values = pd.concat([headline,not_headline],axis=0)

else:
  # After sorting, calculate the headline CPI by taking the sum of products of values and weights for each subject
  not_headline = submission_values[submission_values['CPI'] != 'Headline_CPI']
  not_headline['CPI'] = not_headline['CPI'].str.lower().str.strip()
  not_headline['weights'] = not_headline['CPI'].map(weights_2023)
  headline = pd.DataFrame(data={'CPI': ['headline CPI'], 'value': [not_headline.apply(lambda row: (row['value']*row['weights']),axis=1).sum()]})
  submission_values = pd.concat([headline,not_headline],axis=0)


# Rename IDs to match the file format shown on zindi
submission_values['ID'] = date_obj.strftime('%B')+'_'+submission_values['CPI']
submission_values['Value'] = submission_values['value']
submission_values = submission_values[['ID','Value']]

<ipython-input-120-5e338fa3bc71>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_headline['CPI'] = not_headline['CPI'].map(cpi_dict)
<ipython-input-120-5e338fa3bc71>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_headline['weights'] = not_headline['CPI'].map(weights_2023)


In [ ]:
validation_predictions.to_csv('april.csv')

In [ ]:
from sklearn.metrics import mean_squared_error

test_data = actual.merge(forecasted, on="date", suffixes=("_actual", "_predicted"))

mse = mean_squared_error(test_data.Value_actual, test_data.Value_predicted, squared=False)  # RMSE
print("The root mean squared error (RMSE) on test set: {:.2f}".format(mse))